In [1]:
import os
import numpy as np 
import pandas as pd 
import SimpleITK as sitk
from tqdm import tqdm_notebook as tqdm

# ref.: https://www.kaggle.com/stainsby/fast-tested-rle
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)
 
def rle_decode(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

In [2]:
path = r"./Segmentation_out/Task101_Lung"
patients = os.listdir(path)


In [10]:
sample_subm_file = pd.read_csv('sampleSubmission.csv')
sample_subm_file.head(5)

,Id,Predicted
0,LUNG1_001,17 1 529 1 1041 1 1553 1 2065 1 2577 1 3089 1 ...
1,LUNG1_002,17 1 529 1 1041 1 1553 1 2065 1 2577 1 3089 1 ...
2,LUNG1_003,17 1 529 1 1041 1 1553 1 2065 1 2577 1 3089 1 ...
3,LUNG1_004,17 1 529 1 1041 1 1553 1 2065 1 2577 1 3089 1 ...
4,LUNG1_005,17 1 529 1 1041 1 1553 1 2065 1 2577 1 3089 1 ...


In [11]:
submission_file = pd.DataFrame(None,columns=sample_subm_file.columns)

for pat in tqdm(patients):
    mask = sitk.ReadImage(os.path.join(path,pat))
    submission_file = submission_file.append({'Id':pat[:-7].replace('_','-'),'Predicted':rle_encode(sitk.GetArrayFromImage(mask))},
                           ignore_index=True)
    

<ipython-input-11-d4c01e2923e9>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for pat in tqdm(patients):


  0%|          | 0/72 [00:00<?, ?it/s]

In [12]:
submission_file

,Id,Predicted
0,LUNG1-351,17177290 1 17436358 1 17436869 5 17437381 7 17...
1,LUNG1-352,19017401 3 19017912 5 19018423 10 19018933 11 ...
2,LUNG1-353,9877856 4 9878368 5 9878880 5 9879392 5 987990...
3,LUNG1-354,
4,LUNG1-355,13534001 3 13534511 6 13535022 7 13535534 8 13...
...,...,...
67,LUNG1-418,18490183 2 18490695 2 18491208 1 18492235 2 18...
68,LUNG1-419,
69,LUNG1-420,14846275 2 14846787 2 14847299 2 14847811 2 15...
70,LUNG1-421,11909298 1 11909808 3 11910320 2 12170930 6 12...


In [13]:
submission_file.to_csv('submission.csv',index=False)